# Import packages

In [ ]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from numpy import array
import tensorflow as tf
import pickle
import keras
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import concatenate
from keras.layers.recurrent import LSTM,GRU
from sklearn.model_selection import TimeSeriesSplit
import time
import itertools
import os
import warnings
warnings.filterwarnings('ignore')

# Helper Funtions

In [ ]:
list_features_withWeather = ['time of day_1.0', 'time of day_2.0', 'time of day_3.0', 'time of day_4.0', 'time of day_5.0', 'time of day_6.0', 
              'time of day_7.0', 'time of day_8.0', 'time of day_9.0', 'time of day_10.0', 'time of day_11.0', 'time of day_12.0', 
              'time of day_13.0', 'time of day_14.0', 'time of day_15.0', 'time of day_16.0', 'time of day_17.0', 'time of day_18.0', 
              'time of day_19.0', 'time of day_20.0', 'time of day_21.0', 'time of day_22.0', 'time of day_23.0', 'time of day_24.0',
              'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7',
              'temp(C)',  'wind_speed(Km/h)', 'rel_humi']

list_features_noWeather = ['time of day_1.0', 'time of day_2.0', 'time of day_3.0', 'time of day_4.0', 'time of day_5.0', 'time of day_6.0', 
              'time of day_7.0', 'time of day_8.0', 'time of day_9.0', 'time of day_10.0', 'time of day_11.0', 'time of day_12.0', 
              'time of day_13.0', 'time of day_14.0', 'time of day_15.0', 'time of day_16.0', 'time of day_17.0', 'time of day_18.0', 
              'time of day_19.0', 'time of day_20.0', 'time of day_21.0', 'time of day_22.0', 'time of day_23.0', 'time of day_24.0',
              'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']  


def normalize(df, column_list):
    result = df.copy()
    # encoding 'time of the day' and 'weekday' to dummies
    result = pd.get_dummies(result, columns = ['time of day', 'weekday'])
    for feature_name in column_list:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


def split_sequence(sequence, n_steps_in, n_steps_out):
    """
    split time series to X and y with predefined n_steps_in, n_steps_out
    for example n_steps_in = 336, n_steps_out = 1 use 336 time points to generate 1-hour-ahead prediction
    ----------
    Parameters:
    sequence : time series
    n_steps_in: length of historical net load input in hours
    n_steps_out: length of net load prediction horizon in hours
    ----------
    return: Input sequence X as array, y sequence as array 
    """
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of the sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are out of range
        if out_end_ix+1 > len(sequence):
            break
        # define input and output sequence
        seq_x, seq_y = sequence[i:end_ix], sequence[out_end_ix:out_end_ix+1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def pinball_loss(q, y, f):
    """
    define loss function
    ----------
    Parameters:
    q: quantile
    y: real observation
    f: forecast
    ----------
    return: loss function 
    """
    e = (y-f)
    return keras.backend.mean(keras.backend.maximum(q*e, (q-1)*e), axis=-1)


def run_model(input_data, n_train, n_steps_in, n_steps_out, weather_included, method, learning_rate, n_unit, n_Dense_unit, n_batch_size, 
              n_epochs, n_test):
    """
    This function first builds the model, trains the model and then tests the model
    ----------
    Parameters:
    input_data: input_data
    n_train: length of training set
    n_steps_in: length of historical net load input in hours
    n_steps_out: length of net load prediction horizon in hours
    weather_included: Boolean indicating whether weather data are to be taken into account 
    method: desired prediction method (GRU, LSTM, Regression)
    n_test: length of test set

    hyperparameters:
    learning_rate: learning rate
    n_unit: amount of neurons in recurrent hidden layers
    n_Dense_unit: amount of neurons in dense hidden layers
    n_batch_size: batch size
    n_epochs: epochs
    
    ----------
    return: 
    yhat: forecast
    train_loss: loss on train set
    test_loss: loss on test set
    train_time: time needed for model training
    """
    
    
    if weather_included == 1:
        X2_feature_list = list_features_withWeather
    else:
        X2_feature_list = list_features_noWeather
    
    # Set X1_train, X2_train, and y_train
    X1_seq = input_data['net_consumption'].values.flatten().tolist()[0:n_train]
    X1_train, y_train =  split_sequence(X1_seq, n_steps_in, n_steps_out)
    X1_train = X1_train.reshape((X1_train.shape[0], X1_train.shape[1], 1))
    X2_train = input_data[X2_feature_list][(n_steps_in + n_steps_out):n_train].values.flatten()
    X2_train = X2_train.reshape((X1_train.shape[0], len(X2_feature_list)))
    
    # build model according to figure 3 in the paper
    if method == "LSTM":
        input1 = Input(shape=(n_steps_in, 1))
        input2 = Input(shape=(len(X2_feature_list),))
        recurrent_hidden1 = LSTM(n_unit, activation='sigmoid', return_sequences=True, bias_regularizer='l2')(input1)
        recurrent_hidden2 = LSTM(n_unit, activation='relu', bias_regularizer='l2')(recurrent_hidden1)
        recurrent_output = Dense(1, activation='sigmoid')(recurrent_hidden2)
        merge = keras.layers.concatenate([recurrent_output, input2])
        hidden1 = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(merge)
        hidden_last = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(hidden1)
    elif method == "GRU":
        input1 = Input(shape=(n_steps_in, 1))
        input2 = Input(shape=(len(X2_feature_list),))
        recurrent_hidden1 = GRU(n_unit, activation='sigmoid', return_sequences=True, bias_regularizer='l2')(input1)
        recurrent_hidden2 = GRU(n_unit, activation='relu', bias_regularizer='l2')(recurrent_hidden1)
        recurrent_output = Dense(1, activation='sigmoid')(recurrent_hidden2)
        merge = keras.layers.concatenate([recurrent_output, input2])
        hidden1 = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(merge)
        hidden_last = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(hidden1)
    elif method == "Regression":
        input1 = Input(shape=(n_steps_in,)) # this differs from GRU and LSTM model
        input2 = Input(shape=(len(X2_feature_list),))
        merge = keras.layers.concatenate([input1, input2])
        hidden1 = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(merge)
        hidden2 = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(hidden1)
        hidden3 = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(hidden2)
        hidden_last = Dense(n_Dense_unit, activation='relu', bias_regularizer='l2')(hidden3)        

    output_10 = Dense(1, activation='sigmoid', name='output_10')(hidden_last)
    output_25 = Dense(1, activation='sigmoid', name='output_25')(hidden_last)
    output_50 = Dense(1, activation='sigmoid', name='output_50')(hidden_last)
    output_75 = Dense(1, activation='sigmoid', name='output_75')(hidden_last)
    output_90 = Dense(1, activation='sigmoid', name='output_90')(hidden_last)
    model = Model(inputs=[input1, input2], outputs=[output_10, output_25, output_50, output_75, output_90])
    
    optimizer = keras.optimizers.Adam(learning_rate = learning_rate, beta_1=0.9,
      beta_2=0.999, epsilon=1e-07,)

    model.compile(loss={'output_10': lambda y,f: pinball_loss(0.10,y,f), 
                        'output_25': lambda y,f: pinball_loss(0.25,y,f),
                        'output_50': lambda y,f: pinball_loss(0.50,y,f),
                        'output_75': lambda y,f: pinball_loss(0.75,y,f),
                        'output_90': lambda y,f: pinball_loss(0.90,y,f)},
                  optimizer=optimizer)
    
    # fit model
    start = time.time()
    history = model.fit([X1_train, X2_train], [y_train, y_train, y_train, y_train, y_train], epochs=n_epochs,  batch_size=n_batch_size, shuffle=False)


    print("fit complete!")
    end = time.time()
    train_time = end - start
    print("train_time is: " + str(train_time) + " seconds") 
    
    train_loss = model.evaluate([X1_train, X2_train], [y_train, y_train, y_train, y_train, y_train])
    print("train_loss is: " + str(train_loss))
    
    # test model
    n_total = input_data.shape[0]
    X1_test_seq = input_data['net_consumption'].values.flatten().tolist()[n_train - n_steps_in - n_steps_out:n_test]
    X1_test, y_test =  split_sequence(X1_test_seq, n_steps_in, n_steps_out)
    X1_test = X1_test.reshape((X1_test.shape[0], X1_test.shape[1], 1))
    X2_test = input_data[X2_feature_list][n_train:n_test].values.flatten()
    X2_test = X2_test.reshape((X1_test.shape[0], len(X2_feature_list)))
    yhat = model.predict([X1_test, X2_test])
    test_loss = model.evaluate([X1_test, X2_test], [y_test, y_test, y_test, y_test, y_test])
    print("test_loss is: " + str(test_loss))
    
    
    return yhat, train_loss, test_loss, train_time

def summarize_result(yhat, id, method, weather_included, n_test, learning_rate, n_Dense_unit, n_unit):
    """
    summarize forecast and corresponding loss 
    ----------
    Parameters:
    yhat: forecast
    id: Household id 
    method: desired forecast method (GRU, LSTM, Regression)
    weather_included: Boolean indicating whether weather data are to be taken into account
    n_test: length of test set

    learning_rate: learning rate
    n_Dense_unit: amount of neurons in dense hidden layers
    n_unit: amount of neurons in recurrent hidden layers
    ----------
    return: 
    result_yhat: forecast
    result_loss: forecast loss
    """
  
    columns_list = ['id', 'method', 'weather_included', 'quantile', 'train_loss', 'test_loss', 'learning_rate', 'n_Dense_unit', 'n_unit']+ list(range(1, n_test-n_train+1))
    result_yhat = pd.DataFrame(columns=columns_list)
    q = ['10%', '25%', '50%', '75%', '90%']
    for i in range(0,5):
        result_list = [id, method, weather_included, q[i], yhat[1][i+1], yhat[2][i+1], learning_rate, n_Dense_unit, n_unit] + yhat[0][i].flatten().tolist()
        result = pd.DataFrame([result_list],columns=columns_list)
        result_yhat= result_yhat.append(result)
        
    columns_list = ['id', 'method', 'weather_included', 'time', 'train_loss', 'test_loss', 'learning_rate', 'n_Dense_unit', 'n_unit']
    result_loss = pd.DataFrame(columns=columns_list)
    result_list = [id, method, weather_included, yhat[3], yhat[1][0], yhat[2][0], learning_rate, n_Dense_unit, n_unit]
    result = pd.DataFrame([result_list],columns=columns_list)
    result_loss= result_loss.append(result)
    return result_yhat, result_loss


def forecast(input_data, id, n_steps_out, n_total, n_train, n_steps_in, n_test, hyper_params, path):
    """
    Performes forecasting, saves results and returns a list of forecast results and list of corresponding losses
    ----------
    Parameters:
    input_data: input_data
    id: Household ids
    n_steps_out: length of net load prediction horizon in hours
    n_total: length of total data set 
    n_train: length of training set
    n_steps_in: length of historical net load input in hours
    n_test: length of test set
    hyper_params: set of hyperparameters
    path: output path
    ----------
    return: 
    yhat_list: list of forecast results
    loss_list: list of corresponding losses 
    """
    input_data = input_data.loc[input_data['id'] == id].head(n_total)
    yhat_list = pd.DataFrame()
    loss_list = pd.DataFrame()
    for method, weather_included, learning_rate, n_Dense_unit, n_unit in hyper_params:
        print ("learning_rate, n_unit, n_Dense_unit: ", learning_rate, n_unit, n_Dense_unit)
        yhat = run_model(input_data, n_train, n_steps_in, n_steps_out, weather_included, method, learning_rate, n_unit, n_Dense_unit, n_batch_size, n_epochs, n_test)
        yhat, loss = summarize_result(yhat, id, method, weather_included, n_test, learning_rate, n_Dense_unit, n_unit)
        yhat_list = yhat_list.append(yhat)
        loss_list = loss_list.append(loss)
        print('\n')
    yhat_list.to_pickle(path+'/yhat_net_'+str(id)+'_'+method+'_'+str(weather_included))
    loss_list.to_pickle(path+'/loss_net_'+str(id)+'_'+method+'_'+str(weather_included))
    return yhat_list, loss_list

In [ ]:
# Read in data 
input_data = pd.read_pickle('ProbabilisticLoadForecasting/data/input/preprocessed_input_data')
id_list = input_data['id'].unique()
id_list = id_list.tolist() # a list with the ids of all (40) houses
id_list = id_list
input_data = normalize(input_data, ['net_consumption', 'temp(C)', 'wind_speed(Km/h)', 'rel_humi'])

Due to the time series character and the limited time range of the data (one year), a **two-fold rolling window approach** is conducted to cross-validate the models. First, we train, validate and test our models only on the first 80% of data, i.e. with a train-validation-test split of 40-20-20. Second, we expand the training window, resulting in a 60-20-20 split. For the final evaluation, we average the test losses from step one and two.




In [ ]:
#this cell calls the forecasting function
#select the desired method, parameters and data set split 

## Data set split
#number of total observations
n_total = len(input_data.index.unique()) 
# number of observations used for training
n_train = int(round(n_total * 0.4, 0)) # for cross-validation with expanding window approach, you can repeat the calculations with other factors e.g.: 0.5, 0.6 
# number of observations used for testing
n_test = int(n_train + round(n_total * 0.2, 0))

## Sliding window
# length of historical net load input in hours
n_steps_in = 336
# length of net load prediction horizon in hours
n_steps_out = 1


##Modelparameters
method = 'GRU' # GRU, LSTM, Regression
weather_included = 1 # 1: withWeather, 0: withoutWeather


##hyperparameters that are not tuned 
n_epochs = 15
n_batch_size = 2048 

##Hyperparameters to tune

hyper_params = [(method, weather_included, 0.001, 10, 4), (method, weather_included, 0.001, 10, 8), (method, weather_included, 0.001, 10, 12),
                (method, weather_included, 0.001, 30, 4), (method, weather_included, 0.001, 30, 8), (method, weather_included, 0.001, 30, 12),
                (method, weather_included, 0.001, 50, 4), (method, weather_included, 0.001, 50, 8), (method, weather_included, 0.001, 50, 12),
                (method, weather_included, 0.01, 10, 4), (method, weather_included, 0.01, 10, 8), (method, weather_included, 0.01, 10, 12),
                (method, weather_included, 0.01, 30, 4), (method, weather_included, 0.01, 30, 8), (method, weather_included, 0.01, 30, 12),
                (method, weather_included, 0.01, 50, 4), (method, weather_included, 0.01, 50, 8), (method, weather_included, 0.01, 50, 12),
                (method, weather_included, 0.1, 10, 4), (method, weather_included, 0.1, 10, 8), (method, weather_included, 0.1, 10, 12),
                (method, weather_included, 0.1, 30, 4), (method, weather_included, 0.1, 30, 8), (method, weather_included, 0.1, 30, 12),
                (method, weather_included, 0.1, 50, 4), (method, weather_included, 0.1, 50, 8), 
                (method, weather_included, 0.1, 50, 12)
                ]

path = '' #Set output path  

for i in id_list:
    print('id: ', i)
    locals()['yhat_net_'+str(i)], locals()['yloss_net_'+str(i)] =  forecast(input_data, i , n_steps_out, n_total, n_train, n_steps_in, n_test, hyper_params, path) 

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
fit complete!
train_time is: 23.626830339431763 seconds
100/100 [==============================] - 5s 41ms/step - loss: 0.3820 - output_10_loss: 0.1334 - output_25_loss: 0.0832 - output_50_loss: 0.0810 - output_75_loss: 0.0570 - output_90_loss: 0.0268
train_loss is: [0.3820449113845825, 0.13341447710990906, 0.083172507584095, 0.08102605491876602, 0.057009004056453705, 0.02680768445134163]
55/55 [==============================] - 2s 38ms/step - loss: 0.2864 - output_10_loss: 0.0992 - output_25_loss: 0.0523 - output_50_loss: 0.0612 - output_75_loss: 0.0492 - output_90_loss: 0.0238
test_loss is: [0.2864362895488739, 0.09924765676259995, 0.05233195796608925, 0.061223652213811874, 0.04923900589346886, 0.02377885952591896]


learning_rate, n_unit, n_Dense_unit:  0.001 12 50
Epoch 1/15
2/2 [==============================] - 6s 689ms/step - loss: 0.7136 - output_10_loss: 0.2529 - output_25_loss: 0.2245 - output_50_loss: 0.1435 

In [ ]:
# read in validation results and identify hyper parameter combination with lowest loss
path =  ('') #set input path

data=[]
for i, file in enumerate(os.listdir(path)):
    if file.startswith('loss'):
        output_file = pd.read_pickle(os.path.join(path,file))
        index=pd.Series(range(0,len(output_file)))
        output_file=output_file.set_index(index)
        min_idx=output_file['test_loss'].idxmin()
        data.append(output_file.iloc[min_idx,])

df_min_test_loss = pd.DataFrame(data)
df_min_test_loss.to_pickle('') #set output path

In [ ]:
##Load the model with the best hyper parameter combination and run it on test set

## Data set split
#number of total observations
n_total = len(input_data.index.unique()) 
# number of observations used for training
n_train = int(round(n_total * 0.8, 0)) # set accordingly
# number of observations used for testing
n_test = int(n_train + round(n_total * 0.2, 0)) # set accordingly

path = '' # set output path

method = 'GRU' # GRU, LSTM, Regression
weather_included = 1 # 1: withWeather, 0: withoutWeather

for i in id_list:
    print('id: ', i)
    df_min_test_loss = pd.read_pickle('') # set input path to min Test Loss Parameters
    df_min_test_loss = df_min_test_loss[df_min_test_loss['id'] == i]
    learning_rate = float(df_min_test_loss['learning_rate'])
    n_Dense_unit = int(df_min_test_loss['n_Dense_unit'])
    n_unit = int(df_min_test_loss['n_unit'])
    hyper_params = [(method, weather_included, learning_rate, n_Dense_unit, n_unit)]
    print('Optimal hyperparams: ', hyper_params)
    locals()['yhat_net_'+str(i)], locals()['yloss_net_'+str(i)] =  forecast(input_data, i , n_steps_out, n_total, n_train, n_steps_in, n_test, hyper_params, path) 

In [ ]:
# read in test results and aggregate them into one file
path = ('') # set input path to test output

data=pd.DataFrame()
for i, file in enumerate(os.listdir(path)):
    if file.startswith('loss'):
        output_file = pd.read_pickle(os.path.join(path,file))
        data = data.append(output_file)

data.to_pickle('') # set output path

In [ ]:
#recombine aggregated test results with customer information for houshold-type specific evaluation

data = pd.read_pickle('')  #set input path to aggragated data
df_customergroups = pd.read_pickle('')  #set input path to initial preprocessed_input_data
df_customergroups = df_customergroups.drop(['weekday', 'time of day',	'wind_speed(Km/h)',	'temp(C)','rel_humi',	'pressfc', 'delivery_service_class'], axis=1)
df_customergroups.net_consumption.describe()
df_customergroups.groupby(['id']).net_consumption.std()
df_customergroups
df_customergroups.reset_index(inplace=True)  
df_customergroups = df_customergroups.set_index(['id'])
df_customergroups
data = data.set_index(['id'])
data = data.join(df_customergroups, how = 'left')
data.to_pickle('') # set output path

# Plotting

In [ ]:
#Plot performance comparison of forecasting methods across customer types
ylosses = pd.read_pickle('') #set input path to combined aggregated results with houshold-types
ylosses = ylosses.groupby(['id', 'household_type', 'method', 'weather_included']).mean('test_loss')

ylosses.reset_index(inplace=True)  
ylosses = ylosses.sort_values(by = 'id')
ylosses = ylosses.set_index(['id'])

#extract losses of each method
ylosses_GRU1 = ylosses.loc[(ylosses['weather_included'] == 1) & (ylosses['method'] == 'GRU')]
ylosses_LSTM1 = ylosses.loc[(ylosses['weather_included'] == 1) & (ylosses['method'] == 'LSTM')]
ylosses_Regression1 = ylosses.loc[(ylosses['weather_included'] == 1) & (ylosses['method'] == 'Regression')]
ylosses_LSTM0 = ylosses.loc[(ylosses['weather_included'] == 0) & (ylosses['method'] == 'LSTM')]

fig, ax = plt.subplots(figsize = (8, 6))
ax.scatter(x = ylosses_LSTM1['test_loss'].values, y = ylosses_GRU1['test_loss'].values, color = '#006d2c', s = 22, alpha = 0.8, marker = 'D', label = 'QLSTM')
ax.scatter(x = ylosses_Regression1['test_loss'].values, y = ylosses_GRU1['test_loss'].values, color = '#cb181d', s = 22, alpha = 0.8, marker = 's', label = 'QREGNN')
ax.scatter(x = ylosses_LSTM0['test_loss'].values, y = ylosses_GRU1['test_loss'].values, color = '#08519c', s = 22, alpha = 0.8, marker = 'v', label = 'QLSTM_noWeather')
ax.plot([0, 0.55], [0, 0.55], c = 'black', label = 'y=x', linewidth = 2)
ax.legend(fontsize = 15)
ax.set_ylim([0, 0.55])
ax.set_xlim([0, 0.55])
ax.set_xlabel('Pinball Loss of benchmark methods $[kWh]$', fontsize = 15)
ax.set_ylabel('Pinball Loss of QGRU $[kWh]$', fontsize = 15)
fig.savefig('') # set output path

In [ ]:
# plot annual net consumption
data = pd.read_pickle('') #set input path to aggragated data
df_customergroups = pd.read_pickle('')  #set input path to initial preprocessed_input_data
df_std = df_customergroups.groupby(['id']).net_consumption.std()
df_std = df_std.set_index('id')


ylosses = pd.read_pickle('') #set input path to combined aggregated results with houshold-types
ylosses = ylosses.groupby(['id', 'household_type', 'method', 'weather_included']).mean(['test_loss'])
ylosses.reset_index(inplace=True)  
ylosses = ylosses.sort_values(by = 'id')
ylosses = ylosses.set_index(['id'])

ylosses = ylosses.join(df_std, how = 'left')

ylosses_GRU1 = ylosses.loc[(ylosses['weather_included'] == 0) & (ylosses['method'] == 'LSTM')]
ylosses_GRU1_nono = ylosses_GRU1.loc[(ylosses_GRU1['household_type'] == 'solar_no_heating_no')]
ylosses_GRU1_noyes = ylosses_GRU1.loc[(ylosses_GRU1['household_type'] == 'solar_no_heating_yes')]
ylosses_GRU1_yesno = ylosses_GRU1.loc[(ylosses_GRU1['household_type'] == 'solar_yes_heating_no')]
ylosses_GRU1_yesyes = ylosses_GRU1.loc[(ylosses_GRU1['household_type'] == 'solar_yes_heating_yes')]


fig, ax = plt.subplots(figsize = (8, 6))
ax.scatter(x = ylosses_GRU1_nono['net_consumption'].values, y = ylosses_GRU1_nono['test_loss'].values, color = '#9ecae1', s = 30, alpha = 1, marker = 's', label = 'Household')
ax.scatter(x = ylosses_GRU1_noyes['net_consumption'].values, y = ylosses_GRU1_noyes['test_loss'].values, color = '#4292c6', s = 30, alpha = 1, marker = 'D', label = 'Household with heating')
ax.scatter(x = ylosses_GRU1_yesno['net_consumption'].values, y = ylosses_GRU1_yesno['test_loss'].values, color = '#08519c', s = 30, alpha = 1, marker = 'v', label = 'Household with solar')
ax.scatter(x = ylosses_GRU1_yesyes['net_consumption'].values, y = ylosses_GRU1_yesyes['test_loss'].values, color = '#08306b', s = 30, alpha = 1, marker = 'o', label = 'Household with heating and solar')
ax.legend(fontsize = 15, loc = 'upper left')
ax.set_ylim([0, 0.7])
ax.set_xlabel('Annual net consumption $[kWh]$', fontsize = 15)
ax.set_ylabel('Average hourly Pinball Loss $[kWh]$', fontsize = 15)
fig.savefig('') # set output path